In [1]:
import numpy as np
import pandas as pd
import gc
import json
import torch
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('InfoDataset/pollution.json', 'r') as json_file:
    data_info = json.load(json_file)
num_name = data_info['num_name']
cat_name = data_info['cat_name']
data = pd.read_csv("CleaningDataset/pollution.csv", index_col=0, parse_dates=True)

In [3]:
def process_num_var(input_data, context_window=96, num_sample=2000, num_feature=7):
    N, M = input_data.shape
    B = N - context_window + 1
    B = num_sample
    
    output_data = np.zeros((B, context_window, num_feature), dtype=np.float32)
    mask = np.zeros((B, context_window, num_feature), dtype=bool)
    
    for i in range(B):
        output_data[i, :, :M] = input_data[i:i+context_window, :].astype(np.float32)
        mask[i, :, :M] = True
    
    return output_data, mask

def process_cat_var(input_data, context_window=96, num_sample=2000):
    N, M, D = input_data.shape
    B = N - context_window + 1
    B = num_sample

    output_data = np.zeros((B, context_window, M, D), dtype=np.float32)
    
    for i in range(B):
        output_data[i] = input_data[i:i+context_window, :, :].astype(np.float32)
    
    return output_data

In [4]:
input_data = data[num_name].values
print(input_data.shape)
output_data, mask = process_num_var(input_data, context_window=96, num_sample=2000, num_feature=7)
print("Output data shape:", output_data.shape)
print("Mask shape:", mask.shape)

(43824, 7)
Output data shape: (2000, 96, 7)
Mask shape: (2000, 96, 7)


In [5]:
np.save('TimeSeriesDataset/num/pollution.npy', output_data)
output_data = np.load('TimeSeriesDataset/num/pollution.npy')
print(output_data.shape)

np.save('TimeSeriesDataset/mask/pollution.npy', mask)
mask = np.load('TimeSeriesDataset/mask/pollution.npy')
print(mask.shape)

(2000, 96, 7)
(2000, 96, 7)


In [6]:
input_data = np.load('TextEmbedding/pollution.npy')
print(input_data.shape)
output_data = process_cat_var(input_data, context_window=96, num_sample=2000)
print(output_data.shape)

(43824, 16, 768)
(2000, 96, 16, 768)


In [7]:
np.save('TimeSeriesDataset/cat/pollution.npy', output_data)
output_data = np.load('TimeSeriesDataset/cat/pollution.npy')
print(output_data.shape)

(2000, 96, 16, 768)
